In [86]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import ast
from ast import literal_eval
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import confusion_matrix
from flatten_json import flatten
import re

In [2]:
#business json is stored as an array
business = []
for line in open ('yelp_academic_dataset_business.json','r'):
    business.append(json.loads(line))

In [3]:
data = pd.json_normalize(business)

In [4]:
data.tail(15)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
209378,TLEXFQf9UT4PV5QJrc3QPw,PC Pool Care,"7735 E Evans Rd, Ste 203",Scottsdale,AZ,85260,33.616084,-111.914113,4.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209379,7HDPnk-3T73gpfPbPY4T2w,Footbar - Golden Plaza,"New Golden Plaza, 328 Highway 7 E, Suite 9",Richmond Hill,ON,L4B 3P7,43.840862,-79.399572,4.5,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209380,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,-112.221142,2.0,106,...,"{'monday': False, 'tuesday': False, 'friday': ...","{'dj': False, 'background_music': False, 'no_m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,NaN,{'dj': False},u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN
209382,fNil19SUfPAPnLQrYnFrGQ,Cheyenne West Animal Hospital,3650 N Buffalo Dr,Las Vegas,NV,89129,36.226339,-115.259958,4.5,124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209384,5d6Z-FjQ3g7TZxIs9mJ9kw,Sunbelt Rentals,2066 E University Dr,Tempe,AZ,85281,33.422682,-111.898841,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209386,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,,Las Vegas,NV,89120,36.076977,-115.244794,5.0,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209387,KdGtDThLduB8Vf-z-qKEFA,Aldo,6455 Macleod Trl SW,Calgary,AB,T2H,50.997571,-114.073369,3.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'attributes.BusinessAcceptsCreditCards',
       'attributes.BikeParking', 'attributes.GoodForKids',
       'attributes.BusinessParking', 'attributes.ByAppointmentOnly',
       'attributes.RestaurantsPriceRange2', 'hours.Monday', 'hours.Tuesday',
       'hours.Wednesday', 'hours.Thursday', 'hours.Friday', 'hours.Saturday',
       'hours.Sunday', 'hours', 'attributes', 'attributes.DogsAllowed',
       'attributes.WiFi', 'attributes.RestaurantsAttire',
       'attributes.RestaurantsTakeOut', 'attributes.NoiseLevel',
       'attributes.RestaurantsReservations',
       'attributes.RestaurantsGoodForGroups', 'attributes.HasTV',
       'attributes.Alcohol', 'attributes.RestaurantsDelivery',
       'attributes.OutdoorSeating', 'attributes.Caters',
       'attributes.WheelchairAccessible', 'attributes.AcceptsInsurance',
       'attribu

In [6]:
data.dtypes

business_id                               object
name                                      object
address                                   object
city                                      object
state                                     object
postal_code                               object
latitude                                 float64
longitude                                float64
stars                                    float64
review_count                               int64
is_open                                    int64
categories                                object
attributes.BusinessAcceptsCreditCards     object
attributes.BikeParking                    object
attributes.GoodForKids                    object
attributes.BusinessParking                object
attributes.ByAppointmentOnly              object
attributes.RestaurantsPriceRange2         object
hours.Monday                              object
hours.Tuesday                             object
hours.Wednesday     

In [7]:
data2 = pd.json_normalize(business, max_level=3)

In [8]:
data2.tail(15)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
209378,TLEXFQf9UT4PV5QJrc3QPw,PC Pool Care,"7735 E Evans Rd, Ste 203",Scottsdale,AZ,85260,33.616084,-111.914113,4.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209379,7HDPnk-3T73gpfPbPY4T2w,Footbar - Golden Plaza,"New Golden Plaza, 328 Highway 7 E, Suite 9",Richmond Hill,ON,L4B 3P7,43.840862,-79.399572,4.5,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209380,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,-112.221142,2.0,106,...,"{'monday': False, 'tuesday': False, 'friday': ...","{'dj': False, 'background_music': False, 'no_m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,NaN,{'dj': False},u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN
209382,fNil19SUfPAPnLQrYnFrGQ,Cheyenne West Animal Hospital,3650 N Buffalo Dr,Las Vegas,NV,89129,36.226339,-115.259958,4.5,124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209384,5d6Z-FjQ3g7TZxIs9mJ9kw,Sunbelt Rentals,2066 E University Dr,Tempe,AZ,85281,33.422682,-111.898841,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209386,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,,Las Vegas,NV,89120,36.076977,-115.244794,5.0,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209387,KdGtDThLduB8Vf-z-qKEFA,Aldo,6455 Macleod Trl SW,Calgary,AB,T2H,50.997571,-114.073369,3.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#df = pd.DataFrame(pd.json_normalize(business))

In [10]:
#convert strings in columns into dictionaries; unsuccessful 

dict_columns = ['attributes.NoiseLevel', 'attributes.RestaurantsAttire', 'attributes.Ambience',
                'attributes.Alcohol', 'attributes.WiFi', 'attributes.BusinessParking', 'attributes.GoodForMeal']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df

data = text_to_dict(data)

In [11]:
#flatten(data['attributes.NoiseLevel']); unsuccessful

In [12]:
def create_attributes(data, dictList):
    
    for dictionaryColumn in dictList:
        
        #the attributes column is a string of dictionaries, so one extra step is taken to convert
        if dictionaryColumn == 'attributes':
            expandedColumns = data[dictionaryColumn].map(eval).apply(pd.Series)
        else:
            expandedColumns = data[dictionaryColumn].apply(pd.Series)
        
        data2 = pd.concat([data.drop(dictionaryColumn,axis=1), 
                   expandedColumns]
                  ,axis=1)
        
        #df.fillna(value='{}',inplace=True)
        
    return data2

In [13]:
data2.tail(15)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
209378,TLEXFQf9UT4PV5QJrc3QPw,PC Pool Care,"7735 E Evans Rd, Ste 203",Scottsdale,AZ,85260,33.616084,-111.914113,4.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209379,7HDPnk-3T73gpfPbPY4T2w,Footbar - Golden Plaza,"New Golden Plaza, 328 Highway 7 E, Suite 9",Richmond Hill,ON,L4B 3P7,43.840862,-79.399572,4.5,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209380,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,-112.221142,2.0,106,...,"{'monday': False, 'tuesday': False, 'friday': ...","{'dj': False, 'background_music': False, 'no_m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,NaN,{'dj': False},u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN
209382,fNil19SUfPAPnLQrYnFrGQ,Cheyenne West Animal Hospital,3650 N Buffalo Dr,Las Vegas,NV,89129,36.226339,-115.259958,4.5,124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209384,5d6Z-FjQ3g7TZxIs9mJ9kw,Sunbelt Rentals,2066 E University Dr,Tempe,AZ,85281,33.422682,-111.898841,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209386,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,,Las Vegas,NV,89120,36.076977,-115.244794,5.0,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209387,KdGtDThLduB8Vf-z-qKEFA,Aldo,6455 Macleod Trl SW,Calgary,AB,T2H,50.997571,-114.073369,3.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)

In [15]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [16]:
#create_attributes

#takes a dataframe as an input, as well as a list of columns that are dictionaries
#takes each column that is a dictionary, and expands it into a series of dummy columns

def create_attributes(df, dictList):
    
    for dictionaryColumn in dictList:
        
        #the attributes column is a string of dictionaries, so one extra step is taken to convert
        if dictionaryColumn == 'attributes':
            expandedColumns = df[dictionaryColumn].map(eval).apply(pd.Series)
        else:
            expandedColumns = df[dictionaryColumn].apply(pd.Series)
        
        df = pd.concat([df.drop(dictionaryColumn,axis=1), 
                   expandedColumns]
                  ,axis=1)
        
        df.fillna(value='{}',inplace=True)
        
    return df

In [17]:
dictList = ['attributes', 'Ambience', 'BestNights', 'BusinessParking','Music','GoodForMeal']

In [18]:
expanded = create_attributes(df[['business_id','attributes']], dictList)

TypeError: eval() arg 1 must be a string, bytes or code object

In [19]:
bussinessattributes = create_attributes(df, dictList)

TypeError: eval() arg 1 must be a string, bytes or code object

In [98]:
business = pd.read_csv('~/Yelp/business.csv')

In [21]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",NaN
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,NaN,"Pets, Pet Services, Pet Groomers",NaN
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726649,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [22]:
expanded = create_attributes(business[['business_id','attributes']], dictList)

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
# solution attempt : https://stackoverflow.com/questions/63876637/how-to-json-normalize-a-column-with-nans

In [99]:
#Case 1
business.attributes = business.attributes.fillna('{}')

In [100]:
business.attributes = business.attributes.apply(literal_eval)

In [101]:
business = business.join(pd.json_normalize(business.attributes)).drop(columns=['attributes'])

In [78]:
business.tail(15)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,BestNights,Music,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours
209378,TLEXFQf9UT4PV5QJrc3QPw,PC Pool Care,"7735 E Evans Rd, Ste 203",Scottsdale,AZ,85260,33.616084,-111.914113,4.0,6,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209379,7HDPnk-3T73gpfPbPY4T2w,Footbar - Golden Plaza,"New Golden Plaza, 328 Highway 7 E, Suite 9",Richmond Hill,ON,L4B 3P7,43.840862,-79.399572,4.5,10,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209380,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,2820 N 75th Ave,Phoenix,AZ,85035,33.479139,-112.221142,2.0,106,...,"{'monday': False, 'tuesday': False, 'friday': ...","{'dj': False, 'background_music': False, 'no_m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,[],{'dj': False},u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN
209382,fNil19SUfPAPnLQrYnFrGQ,Cheyenne West Animal Hospital,3650 N Buffalo Dr,Las Vegas,NV,89129,36.226339,-115.259958,4.5,124,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209384,5d6Z-FjQ3g7TZxIs9mJ9kw,Sunbelt Rentals,2066 E University Dr,Tempe,AZ,85281,33.422682,-111.898841,5.0,5,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209386,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,NaN,Las Vegas,NV,89120,36.076977,-115.244794,5.0,217,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209387,KdGtDThLduB8Vf-z-qKEFA,Aldo,6455 Macleod Trl SW,Calgary,AB,T2H,50.997571,-114.073369,3.5,3,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#trying to convert the normalized columns that need further expansion into a dictionary since it seems like it not stored as a dictionary. Dictionary input is require for flatten packaged.
#first fill na values to avoid error with literal_eval in second step
#df.col_str = df.col_str.apply(literal_eval)

#attempt Case 1 with BestNights column; unsuccessful
#business.BestNights = business.BestNights.fillna('{}')
#business.BestNights = business.BestNights.apply(literal_eval)

In [45]:
#Case 1 attempt step 2, BestNights
#business.BestNights = business.BestNights.apply(literal_eval)

In [47]:
#Case 1 attempt step 3, BestNights
#business = business.join(pd.json_normalize(business.BestNights)).drop(columns=['BestNights'])

AttributeError: 'NoneType' object has no attribute 'items'

In [53]:
#Case 2 attempt, BestNights; unsuccessful, 'str' object has no attribute 'values'
#business.BestNights = business.BestNights.fillna({i: {} for i in df.index})
#business = business.join(pd.json_normalize(business.BestNights)).drop(columns=['BestNights'])

In [60]:
#Case 3 attempt, BestNights
business.BestNights = business.BestNights.fillna('[]')

In [61]:
#converts to a list object?
business.BestNights = business.BestNights.apply(literal_eval)

In [70]:
business = business.explode('BestNights').reset_index(drop=True)

KeyError: 0

In [69]:
business.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,BestNights,Music,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726649,4.5,26,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
type(business.iloc[40, 0])

str

In [39]:
flatten(business['attributes']) #unsuccessful, says requires dictionary input

AssertionError: flatten requires a dictionary input

In [80]:
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.BestNights,attributes.Music,attributes.Smoking,attributes.DietaryRestrictions,attributes.DriveThru,attributes.HairSpecializesIn,attributes.BYOBCorkage,attributes.AgesAllowed,attributes.RestaurantsCounterService,attributes.Open24Hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Using solution: https://stackoverflow.com/questions/39169718/convert-string-to-dict-then-access-keyvalues-how-to-access-data-in-a-class

In [81]:
def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"').replace('u"', '"')
    return json.loads(dict_string)

In [83]:
business.BestNights = business.BestNights.apply(string_to_dict)

AttributeError: 'list' object has no attribute 'replace'

In [84]:
def get_lat_lon(dict_string):
    # Get the coordinates string with regex
    rs = re.search("(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)", dict_string).group()
    # Cast to floats
    att = [float(x) for x in rs.split(',')]
    return att

In [87]:
business.BestNights = business.BestNights.apply(get_lat_lon)

TypeError: expected string or bytes-like object

In [ ]:
# trying 2nd solution from here: https://stackoverflow.com/questions/39169718/convert-string-to-dict-then-access-keyvalues-how-to-access-data-in-a-class

In [102]:
Music = business['Music'].astype('str')

In [105]:
#must fill nans, combining previous normalize_json solution from above
Music = business['Music'].fillna('{}')

In [106]:
Music

0         {}
1         {}
2         {}
3         {}
4         {}
          ..
209388    {}
209389    {}
209390    {}
209391    {}
209392    {}
Name: Music, Length: 209393, dtype: object

In [107]:
Music = business['Music'].apply(lambda x: ast.literal_eval(x))

In [109]:
Music = Music.apply(pd.Series)

In [110]:
Music

,dj,background_music,no_music,jukebox,live,video,karaoke
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
209388,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209391,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
Music.to_csv('~/Yelp/attribute_music.csv', index = True)

In [97]:
business

0         nan
1         nan
2         nan
3         nan
4         nan
         ... 
209388    nan
209389    nan
209390    nan
209391    nan
209392    nan
Name: Music, Length: 209393, dtype: object

In [113]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'BusinessAcceptsCreditCards', 'BikeParking',
       'GoodForKids', 'BusinessParking', 'ByAppointmentOnly',
       'RestaurantsPriceRange2', 'DogsAllowed', 'WiFi', 'RestaurantsAttire',
       'RestaurantsTakeOut', 'NoiseLevel', 'RestaurantsReservations',
       'RestaurantsGoodForGroups', 'HasTV', 'Alcohol', 'RestaurantsDelivery',
       'OutdoorSeating', 'Caters', 'WheelchairAccessible', 'AcceptsInsurance',
       'RestaurantsTableService', 'Ambience', 'GoodForMeal', 'HappyHour',
       'BusinessAcceptsBitcoin', 'BYOB', 'Corkage', 'GoodForDancing',
       'CoatCheck', 'BestNights', 'Music', 'Smoking', 'DietaryRestrictions',
       'DriveThru', 'HairSpecializesIn', 'BYOBCorkage', 'AgesAllowed',
       'RestaurantsCounterService', 'Open24Hours'],
      dtype='object')

In [ ]:
#Flatten BusinessParking

In [135]:
BusinessParking = business['BusinessParking'].astype('str')

In [136]:
BusinessParking

0         {'garage': False, 'street': False, 'validated'...
1                                                       nan
2                                                       nan
3         {'garage': False, 'street': False, 'validated'...
4                                                       nan
                                ...                        
209388    {'garage': False, 'street': False, 'validated'...
209389    {'garage': False, 'street': False, 'validated'...
209390    {'garage': False, 'street': False, 'validated'...
209391                                                  nan
209392                                                  nan
Name: BusinessParking, Length: 209393, dtype: object

In [137]:
BusinessParking = business['BusinessParking'].fillna('{}')

In [138]:
BusinessParking

0         {'garage': False, 'street': False, 'validated'...
1                                                        {}
2                                                        {}
3         {'garage': False, 'street': False, 'validated'...
4                                                        {}
                                ...                        
209388    {'garage': False, 'street': False, 'validated'...
209389    {'garage': False, 'street': False, 'validated'...
209390    {'garage': False, 'street': False, 'validated'...
209391                                                   {}
209392                                                   {}
Name: BusinessParking, Length: 209393, dtype: object

In [140]:
BusinessParking = BusinessParking.apply(lambda x: ast.literal_eval(x))

In [141]:
BusinessParking = BusinessParking.apply(pd.Series)

In [144]:
BusinessParking

,garage,street,validated,lot,valet
0,False,False,False,True,False
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,False,False,False,True,False
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
209388,False,False,False,False,False
209389,False,False,False,True,False
209390,False,False,False,True,False
209391,NaN,NaN,NaN,NaN,NaN


In [191]:
BusinessParking.to_csv('~/Yelp/attribute_parking.csv', index = True)

In [ ]:
#RestaurantsAttire

In [145]:
RestaurantsAttire = business['RestaurantsAttire'].astype('str')

In [146]:
RestaurantsAttire = business['RestaurantsAttire'].fillna('{}')

In [147]:
RestaurantsAttire = RestaurantsAttire.apply(lambda x: ast.literal_eval(x))

In [148]:
RestaurantsAttire = RestaurantsAttire.apply(pd.Series)

In [192]:
RestaurantsAttire.to_csv('~/Yelp/attribute_attire.csv', index = True)

In [151]:
#export business to csv with index to join attribute csvs later
business.to_csv('~/Yelp/attribute_business.csv', index = True)

In [ ]:
#NoiseLevel

In [190]:
business['NoiseLevel']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
209388      u'quiet'
209389           NaN
209390    u'average'
209391           NaN
209392           NaN
Name: NoiseLevel, Length: 209393, dtype: object

In [152]:
NoiseLevel = business['NoiseLevel'].astype('str')

In [153]:
NoiseLevel = business['NoiseLevel'].fillna('{}')

In [154]:
NoiseLevel = NoiseLevel.apply(lambda x: ast.literal_eval(x))

In [155]:
NoiseLevel = NoiseLevel.apply(pd.Series)

In [189]:
NoiseLevel

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
209388,quiet
209389,NaN
209390,average
209391,NaN


In [156]:
NoiseLevel.to_csv('~/Yelp/attribute_noise.csv', index = True)

In [ ]:
#Alcohol

In [157]:
#actually not neccessary to convert to string as step 1 so I have skipped it going forward
Alcohol = business['Alcohol'].fillna('{}')

In [158]:
Alcohol = Alcohol.apply(lambda x: ast.literal_eval(x))

In [159]:
Alcohol = Alcohol.apply(pd.Series)

In [160]:
Alcohol.to_csv('~/Yelp/attribute_alcohol.csv', index = True)

In [ ]:
#Ambience

In [161]:
Ambience = business['Ambience'].fillna('{}')

In [162]:
Ambience = Ambience.apply(lambda x: ast.literal_eval(x))

In [163]:
Ambience = Ambience.apply(pd.Series)

In [164]:
Ambience.to_csv('~/Yelp/attribute_ambience.csv', index = True)

In [ ]:
#GoodForMeal

In [165]:
GoodForMeal = business['GoodForMeal'].fillna('{}')

In [166]:
GoodForMeal = GoodForMeal.apply(lambda x: ast.literal_eval(x))

In [167]:
GoodForMeal = GoodForMeal.apply(pd.Series)

In [ ]:
GoodForMeal.to_csv('~/Yelp/attribute_goodformeal.csv', index = True)

In [ ]:
#BestNights

In [169]:
#I think in a prior attempt, I filled nans with [] for BestNights so this step probably doesnt do anything. I will see if it causes problems in the next steps.
BestNights = business['BestNights'].fillna('{}')

In [170]:
BestNights = BestNights.apply(lambda x: ast.literal_eval(x))

In [171]:
BestNights = BestNights.apply(pd.Series)

In [172]:
BestNights

,monday,tuesday,friday,wednesday,thursday,sunday,saturday
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
209388,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209391,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
BestNights.to_csv('~/Yelp/attribute_bestnights.csv', index = True)

In [ ]:
#DietaryRestrictions

In [174]:
DietaryRestrictions = business['DietaryRestrictions'].fillna('{}')

In [175]:
DietaryRestrictions = DietaryRestrictions.apply(lambda x: ast.literal_eval(x))

In [176]:
DietaryRestrictions = DietaryRestrictions.apply(pd.Series)

In [177]:
DietaryRestrictions.to_csv('~/Yelp/attribute_dietrestrictions.csv', index = True)

In [ ]:
#WiFi

In [178]:
WiFi = business['WiFi'].fillna('{}')

In [179]:
WiFi = WiFi.apply(lambda x: ast.literal_eval(x))

In [180]:
WiFi = WiFi.apply(pd.Series)

In [181]:
WiFi.to_csv('~/Yelp/attribute_wifi.csv', index = True)

In [ ]:
#Smoking

In [182]:
Smoking = business['Smoking'].fillna('{}')

In [183]:
Smoking = Smoking.apply(lambda x: ast.literal_eval(x))

In [184]:
Smoking = Smoking.apply(pd.Series)

In [ ]:
Smoking.to_csv('~/Yelp/attribute_smoking.csv', index = True)

In [ ]:
#BYOBCorkage

In [185]:
BYOBCorkage = business['BYOBCorkage'].fillna('{}')

In [186]:
BYOBCorkage = BYOBCorkage.apply(lambda x: ast.literal_eval(x))

In [187]:
BYOBCorkage = BYOBCorkage.apply(pd.Series)

In [188]:
BYOBCorkage.to_csv('~/Yelp/attribute_byobcorkage.csv', index = True)

In [ ]:
#Wifi, alcohol, attire, and byob corkage columns did not fully flatten to dummy columns so I will use the get dummies function to further expand then export to their respective csvs

In [196]:
WiFi = pd.get_dummies(WiFi)
WiFi

,0_free,0_no,0_paid
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
209388,1,0,0
209389,0,0,0
209390,0,1,0
209391,0,0,0


In [197]:
WiFi.to_csv('~/Yelp/attribute_wifi.csv', index = True)

In [198]:
RestaurantsAttire = pd.get_dummies(RestaurantsAttire)

In [199]:
RestaurantsAttire

,0_casual,0_dressy,0_formal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
209388,1,0,0
209389,0,0,0
209390,1,0,0
209391,0,0,0


In [200]:
RestaurantsAttire.to_csv('~/Yelp/attribute_attire.csv', index = True)

In [201]:
Alcohol = pd.get_dummies(Alcohol)

In [202]:
Alcohol

,0_beer_and_wine,0_full_bar,0_none
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
209388,1,0,0
209389,0,0,0
209390,0,0,1
209391,0,0,0


In [203]:
Alcohol.to_csv('~/Yelp/attribute_alcohol.csv', index = True)

In [204]:
BYOBCorkage= pd.get_dummies(BYOBCorkage)

In [205]:
BYOBCorkage

,0_no,0_yes_corkage,0_yes_free
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
209388,0,0,0
209389,0,0,0
209390,0,0,0
209391,0,0,0


In [206]:
BYOBCorkage.to_csv('~/Yelp/attribute_byobcorkage.csv', index = True)

In [209]:
business['categories']

0         Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...
1         Health & Medical, Fitness & Instruction, Yoga,...
2                          Pets, Pet Services, Pet Groomers
3         Hardware Stores, Home Services, Building Suppl...
4         Home Services, Plumbing, Electricians, Handyma...
                                ...                        
209388                    Japanese, Sushi Bars, Restaurants
209389    Department Stores, Food, Mobile Phones, Fashio...
209390    American (New), Food, Burgers, Restaurants, Fa...
209391                     Pet Services, Pet Training, Pets
209392    Tax Services, Professional Services, Accountan...
Name: categories, Length: 209393, dtype: object

In [ ]:
#Categories

In [210]:
def expand_categories(df, cat_var, key):
    all_cats = df[cat_var].str.cat(sep=', ')
    all_cats = all_cats.replace('[', '')
    all_cats = all_cats.replace(']', '')
    all_cats = all_cats.replace("\'","")
    all_cats = all_cats.replace('"','')
    all_cats_list = all_cats.split(', ')
    unique_cats = list(set(all_cats_list))
    unique_cats.remove('Restaurants')
    unique_cats.remove('Food')
    df_cats = pd.DataFrame(index=df[key], columns=unique_cats, data=False)
    df_out = df.merge(df_cats, how='left', left_on=key, right_index=True)
    for cat in unique_cats:
        df_out[cat] = df_out[cat_var].str.contains(cat)
    return df_out  

In [211]:
businesscat = expand_categories(business, 'categories', 'business_id')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [212]:
businesscat

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Private Jet Charter,Pool Cleaners,Candle Stores,Cycling Classes,Plumbing,Court Reporters,Car Rental,Taxis,Auto Insurance,Apartment Agents
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,...,False,False,False,False,False,False,False,False,False,False
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,...,False,False,False,False,False,False,False,False,False,False
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,...,False,False,False,False,False,False,False,False,False,False
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,...,False,False,False,False,False,False,False,False,False,False
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726649,4.5,26,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,False,False,False,False,False,False,False,False,False,False
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,3240 Wilkinson Blvd,Charlotte,NC,28208,35.226307,-80.893539,2.0,13,...,False,False,False,False,False,False,False,False,False,False
209390,SYa2j1boLF8DcGVOYfHPcA,Five Guys,7014-590 E Camelback Rd,Scottsdale,AZ,85251,33.504062,-111.929431,3.5,97,...,False,False,False,False,False,False,False,False,False,False
209391,RSSIsgO00OuWQTRoITacpA,Indian Trail Dog Training,NaN,Indian Trail,NC,28079,35.101965,-80.599385,5.0,3,...,False,False,False,False,False,False,False,False,False,False


In [213]:
#csv with every row flattened minus hours
businesscat.to_csv('~/Yelp/businesscat.csv', index = True)

In [214]:
#creating True/False column for whether or not a business is a restaurant
business['Restaurant'] = business['categories'].str.contains('Restaurants', regex = False)

In [215]:
Restaurants = business[business['Restaurant']==True]

In [216]:
Restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Music,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,{'dj': False},u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [220]:
Restaurants['categories']

8         Ethnic Food, Food Trucks, Specialty Food, Impo...
20               Food, Restaurants, Grocery, Middle Eastern
24                   Restaurants, Cheesesteaks, Poutineries
25             Japanese, Fast Food, Food Court, Restaurants
30        Persian/Iranian, Turkish, Middle Eastern, Rest...
                                ...                        
209381       Nightlife, Diners, Bars, Southern, Restaurants
209383    Restaurants, Sandwiches, Food, Breakfast & Bru...
209385    Restaurants, Pizza, Sandwiches, Fast Food, Chi...
209388                    Japanese, Sushi Bars, Restaurants
209390    American (New), Food, Burgers, Restaurants, Fa...
Name: categories, Length: 63944, dtype: object

In [236]:
#Gives restaurant with the longest category; this did not work
length = 0
iloc = 0

for i in range(0, len(Restaurants)):
    if len(Restaurants['categories'].iloc[i]) > length:
        length = len(Restaurants['categories'].iloc[i])
        iloc = i

Restaurants['categories'].iloc

In [241]:
#create a column that counts the number of categories for each restaurant. Count on the commas and then add 1

Restaurants['Num_Categories'] = Restaurants['categories'].str.count(',')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [242]:
Restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2


In [253]:
Restaurants['Num_Categories'] = Restaurants['Num_Categories'].astype(int)+1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [254]:
Restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3


In [231]:
Restaurants.categories.iloc[0]

'Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Food, Restaurants, Empanadas'

In [232]:
#highest number of categories for a single restaurant
Restaurants.Num_Categories.max()

36

In [244]:
#the restaurant with the highest number of categories, 36
Restaurants.loc[Restaurants['Num_Categories'] == 36]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
20692,IjsLANGkmAqCsF6-zgIA8w,Best Of The Best DJ's,"7322 S Rainbow Blvd, Ste 117",Las Vegas,NV,89139,36.055128,-115.242644,4.0,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,36


In [248]:
#list the categories for that restaurant
Restaurants.categories[Restaurants['Num_Categories'] == 36]

20692    Bakeries, Cafes, DJs, Home Services, Music Ven...
Name: categories, dtype: object

In [255]:
#flatten the categories column for all restaurants, then I will check to see if any category dummy columns need to be dropped
restaurantcat = expand_categories(Restaurants, 'categories', 'business_id')

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [256]:
restaurantcat

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Czech/Slovakian,Boat Repair,Auto Insurance,Pan Asian,Coffee & Tea Supplies,Sandwiches,Pool Halls,Sri Lankan,Adult Education,Fur Clothing
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,False,False,False,False,False,False,False,False,False,False
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,False,False,False,False,False,False,False,False,False,False
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,False,False,False,False,False,False,False,False,False,False
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,False,False,False,False,False,False,False,False,False,False
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,False,False,False,False,False,False,False,False,False,False
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,False,False,False,False,False,True,False,False,False,False
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,False,False,False,False,False,True,False,False,False,False
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,False,False,False,False,False,False,False,False,False,False


In [257]:
restaurantcat.to_csv('~/Yelp/restaurantcat.csv', index = True)

In [262]:
Restaurants.loc[Restaurants['categories'].str.contains('American (New)', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
46,xFc50drSPxXkcLvX5ygqrg,Boomer's Sweet Home Chicago,"5932 W Bell Rd, Ste D-109",Glendale,AZ,85308,33.640391,-112.187028,3.5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
61,-qjn24n8HYF6It9GQrQntw,Five Guys Burgers and Fries,2130 E Arbors Dr,Charlotte,NC,28262,35.338070,-80.757397,4.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
233,wU3wVPC2RQLS_zW-pe83tw,Barrel Lodge Bar & Grill,9700 Page Rd,Streetsboro,OH,44241,41.254650,-81.326077,3.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
441,J7fekWie4ZwR03AZVBisFw,Cafe South,7901 Forest Pine Dr,Charlotte,NC,28273,35.136278,-80.905435,4.0,228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
462,vPWvyl9gxmuOIXDBL6lSpw,She by Morton's,3720 S Las Vegas Blvd,Las Vegas,NV,89158,36.107825,-115.175532,3.5,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209276,2auigv5pF08RjJ8CL1dIVw,O&B Canteen,330 King Street W,Toronto,ON,M5V 3X2,43.646693,-79.390210,3.0,291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6
209281,8WIcKHx7of-pyj2C3CyuIg,Five Guys,9826 Gilead Rd,Huntersville,NC,28078,35.406267,-80.864353,4.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
209292,_JA3TLF5JJ95mthVwEdFuw,Steak ’n Shake,1926 Sardis Rd N,Charlotte,NC,28270,35.140920,-80.732982,2.0,88,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,7
209303,k0Bj4ZYyoICz6Mhx2f9Udg,River Dog Cafe,19900 Detroit Rd,Rocky River,OH,44116,41.480061,-81.841934,4.0,95,...,'no',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,13


In [263]:
Restaurants.loc[Restaurants['categories'].str.contains('Beach Bars', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
130351,vATbGMxLd9cZ3VY_GZAIJw,Cariba Charlie's,"4835 S Rainbow Blvd, Ste 303",Las Vegas,NV,89103,36.100872,-115.244532,4.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
189500,xGFJ_Uutj9iaH8LupRvF6A,MVP's Raw Bar & Grill,10916 Black Dog Ln,Charlotte,NC,28214,35.336302,-80.961875,2.5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,12


In [264]:
Restaurants.loc[Restaurants['categories'].str.contains('Hainan', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
86993,CIFjkjllhTPGyx9eWBwJGQ,Sprout Hainan Noodle,"5 Northtown Way, Unit 14",Toronto,ON,M2N 7A1,43.77486,-79.413522,3.5,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
204940,klWcP_4OVXLStd3VovuA1A,Greedys Restaurant,4338 E 71st St,Cleveland,OH,44105,41.44076,-81.639619,3.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5


In [265]:
Restaurants.loc[Restaurants['categories'].str.contains('Udon', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
59198,C8_zdU7zGLUK3uC4e5AepQ,MeNami,5469 Yonge Street,Toronto,ON,M2N 5S1,43.77784,-79.41477,4.0,462,...,u'no',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7


In [266]:
Restaurants.loc[Restaurants['categories'].str.contains('Swiss Food', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
7340,XPduWZA2t7ELlMHWjv4J9Q,Swiss Chalet Rotisserie & Grill,2 County Court Blvd,Brampton,ON,L6W 3W8,43.662916,-79.727801,3.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
74669,GzF4lVSh61sayvwVMp8RmA,Swiss Chalet Rotisserie & Grill,2990 Eglinton Ave E,Toronto,ON,M1J 2E4,43.740597,-79.232747,2.5,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5


In [267]:
Restaurants.loc[Restaurants['categories'].str.contains('Tuscan', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
4193,QCCVxVRt1amqv0AaEWSKkg,Esther's Kitchen,1130 S Casino Center Blvd,Las Vegas,NV,89104,36.157522,-115.152852,4.5,704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,10
12198,NnOAwa0A2tNPnZl9IHVLPQ,Zeppe's Bistro & Pizzeria,5843 Darrow Rd,Hudson,OH,44236,41.224050,-81.439920,3.5,83,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,13
29441,M4Bi2eyelIP9YQuwD7a3JA,Zia Pia imports + Italian kitchen,224 E 7th St,Charlotte,NC,28202,35.227759,-80.838199,3.5,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
29765,UYh1N1xQhOh-a7nX92xPzQ,Brio Tuscan Grille,6653 Las Vegas Blvd,Las Vegas,NV,89119,36.068152,-115.175930,4.0,1388,...,u'no',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
141966,7RidnmyYyH0cfcX2x7buag,Siena Bistro,10575 Siena Monte Ave,Las Vegas,NV,89135,36.107854,-115.324773,4.0,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,9


In [268]:
Restaurants.loc[Restaurants['categories'].str.contains('Scottish', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
32929,mFYEB6CrMP28fjL_wg4aYw,"Bobby's Place, Olde World Tavern",11440 Braeside Drive SW,Calgary,AB,T2W 3E5,50.953130,-114.115893,4.0,11,...,u'outdoor',NaN,False,NaN,NaN,NaN,NaN,NaN,True,7
42322,VZOQ2-cUP8bfCn7Q7ix9HQ,McGugan's,1058 Gerrard Street E,Toronto,ON,M4M 1Z8,43.669256,-79.335902,3.5,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,9
58172,FfJD4vO-iYUL2Kr0WWzNYg,The Caledonian,856 College Street,Toronto,ON,M6H 1A2,43.654402,-79.423591,4.0,137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
108466,YmVcOfuPXRmyse8Rq7S6vQ,Molly MacPherson's Scottish Pubs & Grill,"127 N Tryon St, Unit 8",Charlotte,NC,28202,35.228365,-80.842652,3.5,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6


In [269]:
Restaurants.loc[Restaurants['categories'].str.contains('Kombucha', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
89958,ewvBhAgfJi43aZmWCd3Zag,Bicycle Nomad,828 N 2nd St,Phoenix,AZ,85004,33.457435,-112.071639,4.5,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,10
93809,9bdVAp-1LlkEKn2eegvCzw,The Witches Brew,8-160 Baldwin Street,Toronto,ON,M5T 3K7,43.655209,-79.399867,4.5,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
187054,JY9MHa3PdjIYp_D7E4GAIw,Wow Wow Hawaiian Lemonade,"6501 E Greenway Pkwy, Ste 152",Scottsdale,AZ,85254,33.624605,-111.941474,4.5,214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,10


In [270]:
Restaurants.loc[Restaurants['categories'].str.contains('Polynesian', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
77695,bAMX1SCQyv09oYOtEYKVjw,Drunk Munk,"7133 E Stetson Dr, Ste 1",Scottsdale,AZ,85251,33.499116,-111.927562,3.5,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
101078,-oSII3bw90cvyLmgsHgmpg,Mokis Hawaiian Grill,"3614 E Southern Ave, Ste 101",Mesa,AZ,85206,33.395472,-111.752960,3.5,249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7


In [271]:
Restaurants.loc[Restaurants['categories'].str.contains('American (Traditional)', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
42,fhNf_sg-XzZ3e7HEVGuOZg,Meat Chix And Wieners,6530 S Decatur Blvd,Las Vegas,NV,89118,36.071196,-115.207463,3.0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
83,7j0kor_fkeYhyEpXh4OpnQ,Scramblers - Strongsville,16726 Pearl Rd,Strongsville,OH,44136,41.295689,-81.835722,3.5,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
104,UFU8ONTkzEkcOk61XJ8JwQ,Nickels,1200 Rue Peel,Montréal,QC,H3B 2T6,45.499435,-73.572117,2.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
108,8nP8ghEpT6WFcM6tfqAaGA,Sugar Bowl,4005 N Scottsdale Rd,Scottsdale,AZ,85251,33.493775,-111.925933,3.5,720,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,True,5
115,8Hvp1tYKiQbBgGIwkCRK5g,Tony's Family Restaurant,1515 W Pleasant Valley Rd,Parma,OH,44134,41.361185,-81.688755,4.0,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209292,_JA3TLF5JJ95mthVwEdFuw,Steak ’n Shake,1926 Sardis Rd N,Charlotte,NC,28270,35.140920,-80.732982,2.0,88,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,7
209303,k0Bj4ZYyoICz6Mhx2f9Udg,River Dog Cafe,19900 Detroit Rd,Rocky River,OH,44116,41.480061,-81.841934,4.0,95,...,'no',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,13
209343,8dK-8e006F_it5bNxTVlaQ,Famous Dave's Bar-B-Que,26410 Great Northern Shopping Ctr,North Olmsted,OH,44070,41.418407,-81.915597,3.5,120,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,True,9
209358,_bvc2j6E8mk_ZiMoh2iMuQ,Sonic Drive-In,6501 W Glendale Ave,Glendale,AZ,85301,33.537861,-112.199338,2.5,13,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,6


In [272]:
Restaurants.loc[Restaurants['categories'].str.contains('Game Meat', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
10851,xlMgaPOpd_99SFTuOtDH0A,Manitoba,271 Rue Saint-Zotique O,Montréal,QC,H2V 1A3,45.528852,-73.616205,4.0,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
39944,xFpjl892AzF_7wH8kfvVWA,Meats Gone Wild,NaN,Las Vegas,NV,NaN,36.035328,-114.985786,4.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
48855,zjtnLXwJvHpXSnAqOkjqww,Food Diva,275 W Warner Rd,Mesa,AZ,85201,33.334739,-111.845354,3.5,3,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,11
81616,X_HMB7QafYd9CAQTQy8aXw,Wild Elk Den,275 W Warner Rd,Chandler,AZ,85225,33.334762,-111.845330,4.5,57,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,12


In [273]:
Restaurants.loc[Restaurants['categories'].str.contains('Rotisserie Chicken', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
118592,dddUz7IweLuFYelc3ZIyNA,Pollo Picante,"635 N Country Club, Ste 3",Mesa,AZ,85201,33.426101,-111.839739,4.0,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
127325,ZTpITvFpX0PQgqNDzZ1DLg,Pollos Culiacan 2,2640 E Tropicana,Las Vegas,NV,89121,36.100148,-115.114315,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7


In [274]:
Restaurants.loc[Restaurants['categories'].str.contains('Slovakian', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
1732,FUCHvOmoHC3YalwRwB97Yg,Kuhar's Carryout,8030 Broadmoor Rd,Mentor,OH,44060,41.652112,-81.377161,4.5,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,10
81349,CGKgDk1f-VAu0BHOivqmLQ,Prague Restaurant,450 Scarborough Golf Club Road,Toronto,ON,M1G 1H1,43.754341,-79.216001,3.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
120390,Y_jBZDRBdILqIUMqiTHgkQ,California Pub & Grill,914 Pape Avenue,East York,ON,M4K 3V2,43.685851,-79.347669,4.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
149111,V01onLi7ezGFSeoaosfw1g,V's Schnitzel House,448 Browns Line,Toronto,ON,M8W 3T9,43.602164,-79.545760,3.5,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
189254,OIB6OB9cgNMh_Q5y5pSKHw,Prague Fine Food Emporium,638 Queen Street W,Toronto,ON,M6J 1E4,43.646936,-79.406030,3.5,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8


In [275]:
Restaurants.loc[Restaurants['categories'].str.contains('Iberian', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
43717,v-2vZKrozNOLX3-DsrjnPg,Dyne,120 Avenue Road,Toronto,ON,M5R 2H4,43.673799,-79.396321,4.0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
83399,s11Ym0QX2Am2uwtnyou4GA,Quinta Restaurant,1282 Dundas Street W,Toronto,ON,M6J 1X7,43.649560,-79.425389,4.0,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6
116456,tr_VNfk6Q0fUfdwrkQEgQQ,Bistro Piquillo,3900 Rue Ethel,Verdun,QC,H4G 1S4,45.464837,-73.567819,4.5,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
138008,a7euZScX4ljHdUBcdwtMvg,Boca Iberica,12 Rue Rachel O,Montréal,QC,H2W 1G1,45.517652,-73.581805,4.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5
185941,TbDd4tvXevqZt_9cwgw9sQ,Azorean Cafe,4715 Liberty Ave,Pittsburgh,PA,15224,40.462067,-79.949227,4.5,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7


In [276]:
Restaurants.loc[Restaurants['categories'].str.contains('Eritrean', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
13007,_4tCeVrLRmI18BscMiHr7Q,Asmara Restaurant,4180 W Desert Inn Rd,Las Vegas,NV,89102,36.130814,-115.196263,4.5,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
198169,ulDxltMssBpGIEkC_A5j3Q,East African Cusine Eritrean And Ethiopian,3200 Monroe Rd,Charlotte,NC,28205,35.203609,-80.801695,4.0,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7


In [277]:
Restaurants.loc[Restaurants['categories'].str.contains('Strip Clubs', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
112330,bbMJNMWjd7n2ztcE79ZwfQ,Blue Moon Gentlemen's Club,2911 E Van Buren St,Phoenix,AZ,85008,33.450866,-112.018800,3.0,20,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6
121266,NUGXuxdv3ef0XGqNsoU4qg,CatHouse,3064 32 Street NE,Calgary,AB,T2Y 6J7,51.079950,-113.987756,1.5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
126578,cNvy9WE3Xk935MGMd36glQ,Pure Gold Club,2630 Windsor Drive,Mississauga,ON,L5J 1K7,43.497468,-79.643281,2.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6
146936,9lDpO8fpgZKd18yskBM3Zw,Diamond Men's Club,1628 Fall St,Cleveland,OH,44113,41.493211,-81.702148,3.0,17,...,u'outdoor',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
154434,GRWIABHedj1m8DDoAuTIMg,Centerfolds Cabaret kept as per website,4416 Paradise Rd,Las Vegas,NV,89169,36.109517,-115.152013,3.5,44,...,u'yes',NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
169221,sF0Y740tAkmG9EeXF90b9Q,Ricks Cabaret Pittsburgh,135 9th St,Pittsburgh,PA,15222,40.444212,-79.998334,3.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6
195504,O589S0opFB9oK8VcL5tRcA,Treasures Gentlemen's Club & Steakhouse,2801 Westwood Dr,Las Vegas,NV,89109,36.140056,-115.173700,3.0,214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,15


In [280]:
Restaurants.loc[Restaurants['categories'].str.contains('Senegalese', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
79639,UKaOWZt_v08yHmDTcIuVdg,Chez Khady,850 Boulevard Décarie,Saint-Laurent,QC,H4L 3L9,45.51210,-73.679789,4.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3
159328,L6_xgiNCxGnekutH54QoGQ,Diolo,232-A Rue Jean-Talon O,Montréal,QC,H2R 2X5,45.53212,-73.620449,4.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3


In [281]:
Restaurants.loc[Restaurants['categories'].str.contains('Pita', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
90695,4jIyp1SzHM5R9EL0SCeXMw,Pita Pit,"4972 South Power Rd, Ste 103",Gilbert,AZ,85234,33.325693,-111.687299,3.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8
187228,ycd3DfIYG-oAiyHXPPZjBA,Pita Pit - Mill Ave,"690 S Mill Ave, Bldg C Ste 121",Tempe,AZ,85281,33.423431,-111.940465,3.0,78,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,True,9


In [282]:
Restaurants.loc[Restaurants['categories'].str.contains('Canadian (New)', regex = False)]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Smoking,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories
220,r3jpH6ZuCQY3o1yBuJ67PQ,Master Mix HotPot,261 Spadina Avenue,Toronto,ON,M5T 1H1,43.652251,-79.397642,3.0,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
831,6AJEPExs2n_8QDZFDj9fkg,Archeo Focacceria,"6301 Silver Dart Drive, Terminal 3",Mississauga,ON,L5P 1B2,43.680461,-79.611733,2.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2
852,nY2oBwH3s5sKox1mERIoVQ,Epicure Cafe & Grill,502 Queen Street W,Toronto,ON,M5V 2B3,43.647741,-79.401279,3.5,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,7
909,-8AoSGOdZyDdTpT2oqX3zQ,Jack Astor's Bar & Grill,"3395, av. des Grandes-Tourelles",Boisbriand,QC,J7H 0A2,45.628706,-73.848171,3.5,15,...,u'outdoor',NaN,False,NaN,NaN,NaN,NaN,NaN,True,8
949,R_nykJJfrfcfeF4Pof34nA,The Guild,1442 Dundas Street W,Toronto,ON,M6J 1Y6,43.649687,-79.430309,3.5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209164,6R4GzPhlFS1n_xCO7U7Gcw,Gayley's Cafe,1424 Dundas Street W,Toronto,ON,M6J 1Y5,43.649680,-79.429522,3.5,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4
209189,WVNp8vVqYOyOML-JWuPsCw,Chez Fabien,778 Rue Saint-Pierre,Terrebonne,QC,J6W 1E4,45.692553,-73.634783,5.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2
209247,_rZyr1lrIoBaz65XiDPP6A,Fixe,5985 Rue Saint-Hubert,Montréal,QC,H2S 2L7,45.534416,-73.598967,4.0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,9
209276,2auigv5pF08RjJ8CL1dIVw,O&B Canteen,330 King Street W,Toronto,ON,M5V 3X2,43.646693,-79.390210,3.0,291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,6


In [285]:
Restaurants['American_Traditional'] = Restaurants['categories'].str.contains('American (Traditional)', regex = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [286]:
Restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,DietaryRestrictions,DriveThru,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories,American_Traditional
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,8,False
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4,False
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3,False
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4,False
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5,False
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5,False
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,5,False
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3,False


In [287]:
Restaurants['American_New'] = Restaurants['categories'].str.contains('American (New)', regex = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [288]:
Restaurants['Canadian_New'] = Restaurants['categories'].str.contains('Canadian (New)', regex = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [289]:
Restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,HairSpecializesIn,BYOBCorkage,AgesAllowed,RestaurantsCounterService,Open24Hours,Restaurant,Num_Categories,American_Traditional,American_New,Canadian_New
8,pQeaRpvuhoEqudo3uymHIQ,The Empanadas House,404 E Green St,Champaign,IL,61820,40.110446,-88.233073,4.5,5,...,NaN,NaN,NaN,NaN,NaN,True,8,False,False,False
20,CsLQLiRoafpJPJSkNX2h5Q,Middle East Deli,4508 E Independence Blvd,Charlotte,NC,28205,35.194894,-80.767442,3.0,5,...,NaN,NaN,NaN,NaN,NaN,True,4,False,False,False
24,eBEfgOPG7pvFhb2wcG9I7w,Philthy Phillys,"15480 Bayview Avenue, unit D0110",Aurora,ON,L4G 7J1,44.010962,-79.448677,4.5,4,...,NaN,NaN,NaN,NaN,NaN,True,3,False,False,False
25,lu7vtrp_bE9PnxWfA8g4Pg,Banzai Sushi,300 John Street,Thornhill,ON,L3T 5W4,43.820492,-79.398466,4.5,7,...,NaN,NaN,NaN,NaN,NaN,True,4,False,False,False
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,...,NaN,NaN,NaN,NaN,NaN,True,5,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209381,xVpE01l6ZXdEtVf5PkRpDg,Julep,829 E Washington Ave,Madison,WI,53703,43.081022,-89.374006,4.0,95,...,NaN,NaN,NaN,NaN,NaN,True,5,False,False,False
209383,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,27045 Lorain Rd,North Olmsted,OH,44070,41.416568,-81.921396,3.0,13,...,NaN,NaN,NaN,NaN,NaN,True,5,False,False,False
209385,hskVqZCPqy-omm9CHi44xQ,Domino's,"6420 Rea Rd, Suite B1",Charlotte,NC,28226,35.078538,-80.818358,2.0,16,...,NaN,NaN,NaN,NaN,NaN,True,5,False,False,False
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,...,NaN,NaN,NaN,NaN,NaN,True,3,False,False,False


In [290]:
Restaurants.to_csv('~/Yelp/americancanadiancats.csv', index = True)